## 1. Import All Packages

In [4]:
%%time
import numpy as np
import math
import pandas as pd
import pickle, os, math
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from random import randint

Wall time: 0 ns


## 2. Read data

The data is combined from three different datasets.

The enermy frequency are respectively 2, 1, 8.

In [5]:
%%time

data_dir = os.path.join(os.curdir, 'Data', 'Score', '400000_68per', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics_0 = ot['data']
target_pics_0 = ot['target']
print(data_pics_0.shape)

data_dir = os.path.join(os.curdir, 'Data', 'Score', '400000_1freq_fires1', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics_3 = ot['data']
target_pics_3 = ot['target']
print(data_pics_3.shape)

data_dir = os.path.join(os.curdir, 'Data', 'Score', '800000_8freq', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics = ot['data']
target_pics = ot['target']
print(data_pics.shape)

data_pics = data_pics.append(data_pics_0).append(data_pics_3)
target_pics = target_pics.append(target_pics_0).append(target_pics_3)


data_pics.reset_index(inplace=True, drop=True)
target_pics.reset_index(inplace=True, drop=True)

print(data_pics.shape)

(90029, 221)
(41569, 221)
(599671, 221)
(731269, 221)
Wall time: 3.44 s


## 3. Preprocess Data

The same preprocessing is done to the scoring training data.

The imbalance between classes are more prominent, so a new group of data is generated using the same method. The new dataset has 82k data points, which should be enough for training purposes.

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    data_pics, target_pics, test_size=0.1, random_state=randint(100, 10000))
print(X_train.shape)
X_train_train, X_vali, y_train_train, y_vali = train_test_split(
    X_train, y_train, test_size=0.3, random_state=randint(100, 10000))
print('class 0 has ' + str(len(y_train.index[y_train[0] == 0].tolist())) + ' points')
print('class 1 has ' + str(len(y_train.index[y_train[0] == 1].tolist())) + ' points')

index_0 = y_train.index[y_train[0] == 0].tolist()
index_1 = y_train.index[y_train[0] != 0].tolist()
index_0_comparable_to_1 = np.random.choice(index_0, math.floor(len(index_1) * 5))
samples = np.concatenate([index_1, index_0_comparable_to_1])
print(str(len(index_1)) + ' + ' + str(len(index_0_comparable_to_1)) + ' = ' + str(len(samples)))


small_data = data_pics.iloc[samples, :]
small_target = target_pics.iloc[samples, :]

X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    small_data, small_target, test_size=0.3, random_state=randint(100, 10000))
X_train_train_small, X_vali_small, y_train_train_small, y_vali_small = train_test_split(
    X_train_small, y_train_small, test_size=0.3, random_state=randint(100, 10000))

y_train_small_m = np.ravel(y_train_small)
y_test_small_m = np.ravel(y_test_small)
y_train_train_small_m = np.ravel(y_train_train_small)
y_vali_small_m = np.ravel(y_vali_small)
y_train_m = np.ravel(y_train)
y_test_m = np.ravel(y_test)
y_train_train_m = np.ravel(y_train_train)
y_vali_m = np.ravel(y_vali)

(658142, 221)
class 0 has 616569 points
class 1 has 41573 points
41573 + 207865 = 249438


## 3. Training

### Multi Layer Perceptor Classifier

In [17]:
mlpc_st = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32, 16, 8),
                        alpha=0.15, max_iter=1000, batch_size=2000,
                        verbose=True, learning_rate_init=0.01, tol=1e-5,
                        learning_rate='adaptive')

mlpc_st.fit(X_train_small, y_train_small_m)
mlpc_st.score(X_test_small, y_test_small_m)

Iteration 1, loss = 0.42540756
Iteration 2, loss = 0.35882314
Iteration 3, loss = 0.34149188
Iteration 4, loss = 0.33233710
Iteration 5, loss = 0.32398253
Iteration 6, loss = 0.32194079
Iteration 7, loss = 0.31699058
Iteration 8, loss = 0.31339209
Iteration 9, loss = 0.31070863
Iteration 10, loss = 0.31005143
Iteration 11, loss = 0.30610510
Iteration 12, loss = 0.30699000
Iteration 13, loss = 0.30644254
Iteration 14, loss = 0.30374718
Iteration 15, loss = 0.30404229
Iteration 16, loss = 0.30242831
Iteration 17, loss = 0.30104619
Iteration 18, loss = 0.30121528
Iteration 19, loss = 0.30139158
Iteration 20, loss = 0.30112993
Training loss did not improve more than tol=0.000010 for two consecutive epochs. Stopping.


0.8849689972204404

In [19]:
ypred = mlpc_st.predict(X_test_small)
np.histogram(ypred - y_test_small_m + 1, bins=[0, 1, 2, 3])

(array([ 6596, 66224,  2012], dtype=int64), array([0, 1, 2, 3]))

In [20]:
sum(ypred) / len(ypred)

0.10595734445157153

In [24]:
from sklearn.metrics import confusion_matrix, f1_score

In [23]:
print(confusion_matrix(y_test_small_m, ypred).ravel())

array([60307,  2012,  6596,  5917], dtype=int64)

In [25]:
print(f1_score(y_test_small_m, ypred))

0.5789061735642306


In [9]:
cross_val_score(mlpc_st, X_train_small, y_train_small_m, cv=5)

Iteration 1, loss = 0.71261470
Iteration 2, loss = 0.55959339
Iteration 3, loss = 0.55142420
Iteration 4, loss = 0.54814739
Iteration 5, loss = 0.54696221
Iteration 6, loss = 0.54093755
Iteration 7, loss = 0.53758349
Iteration 8, loss = 0.53550233
Iteration 9, loss = 0.52066649
Iteration 10, loss = 0.51307636
Iteration 11, loss = 0.50353729
Iteration 12, loss = 0.49809278
Iteration 13, loss = 0.49306400
Iteration 14, loss = 0.48838948
Iteration 15, loss = 0.47812446
Iteration 16, loss = 0.47741668
Iteration 17, loss = 0.47061686
Iteration 18, loss = 0.47172241
Iteration 19, loss = 0.46214790
Iteration 20, loss = 0.46251245
Iteration 21, loss = 0.46349227
Iteration 22, loss = 0.45994913
Iteration 23, loss = 0.45249831
Iteration 24, loss = 0.44145668
Iteration 25, loss = 0.44242797
Iteration 26, loss = 0.43816604
Iteration 27, loss = 0.43772209
Iteration 28, loss = 0.43178104
Iteration 29, loss = 0.42987710
Iteration 30, loss = 0.42212287
Iteration 31, loss = 0.41446887
Iteration 32, los

Iteration 47, loss = 0.35438992
Iteration 48, loss = 0.34580684
Iteration 49, loss = 0.35849859
Iteration 50, loss = 0.35167317
Iteration 51, loss = 0.33723722
Iteration 52, loss = 0.33981456
Iteration 53, loss = 0.33964535
Iteration 54, loss = 0.34691387
Training loss did not improve more than tol=0.000010 for two consecutive epochs. Stopping.


array([0.75629891, 0.74191606, 0.75240798, 0.75068799, 0.756429  ])

In [26]:
import joblib
joblib.dump(mlpc_st, os.path.join(os.curdir,'Models', 'model_score_75_method_1.joblib'))
mlpc_st_loaded = joblib.load(os.path.join(os.curdir,'Models', 'model_score_75_method_1.joblib'))

In [27]:
mlpc_st_loaded.predict([[i for i in range(221)]])

array([0], dtype=int64)

### SVC with scikit-learn

To use SVC, we have to scale down the dataset.

In [28]:
index_0 = y_train.index[y_train[0] == 0].tolist()
index_1 = y_train.index[y_train[0] != 0].tolist()

index_1_svc = np.random.choice(index_1, math.floor(len(index_1) * 0.02))
index_0_svc = np.random.choice(index_0, math.floor(len(index_1_svc) * 5))
samples_svc = np.concatenate([index_1_svc, index_0_svc])
print(str(len(index_1_svc)) + ' + ' + str(len(index_0_svc)) + ' = ' + str(len(samples_svc)))


svc_data = data_pics.iloc[samples_svc, :]
svc_target = target_pics.iloc[samples_svc, :]

X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(
    svc_data, svc_target, test_size=0.3, random_state=randint(100, 10000))
X_train_train_svc, X_vali_svc, y_train_train_svc, y_vali_svc = train_test_split(
    X_train_svc, y_train_svc, test_size=0.3, random_state=randint(100, 10000))

y_train_svc_m = np.ravel(y_train_svc)
y_test_svc_m = np.ravel(y_test_svc)
y_train_train_svc_m = np.ravel(y_train_train_svc)
y_vali_svc_m = np.ravel(y_vali_svc)

831 + 4155 = 4986


In [ ]:
clf_nf = SVC(C=1.0, gamma='auto', verbose=True)
clf_nf.fit(X_train_svc, y_train_svc_m)
clf_nf.score(X_test_svc, y_test_svc_m)

[LibSVM]

## 4. Using predictors in game

### 4.1 without using survival training

There are mainly two parameters that influences the final score of the game.

1. The total number of fires
2. The total number of movements (which is proportional to the length of the game, because for each frame, the fighter moves one step. The steps include not moving.)

The score is definitely proportional to the number of fires and the number of movements.

Score (S) $\propto$ Fires (F), $\propto$ Movs (M)

1. One thing about number of fires, because the training stage the number of fires are set, however in the predicting game stage the number of fires are random. So to eliminate the effect of number of fires on the performance, the best way to check the performance of the game, is to calculate the score-f-m ratio

$$R_{fm} = \frac{S}{F \times M}$$

    Higher score-f-m ratio means better performance.

2. The second method is the score-f ratio, or so-called Hit Rate

$$R_{f} = \frac{S}{F}$$

    Higher score-f ratio means higher hit rate.

<p style="text-align: center;">**enemy frequency: 1 / 1**</p>

Before Score training (6 test runs):

|                    |    1    |    2    |    3    |    4    |    5    |    6    |Average|
|--------------------|--------:|--------:|--------:|--------:|--------:|--------:|---:|
|Lives Consumed      | 2950.000| 2836.000| 2704.000| 2526.000| 2677.000| 2758.000|2741.83
|Score               | 1586.000| 1505.000| 1496.000| 1317.000| 1413.000| 1560.000|1479.5
|Fires               | 3652.000| 3743.000| 3851.000| 3709.000| 3653.000| 3762.000|3728.33
|Moves               |15001.000|15001.000|15001.000|15001.000|15001.000|15001.000|15001
|Ratio(Lives / Mov)  |    0.197|    0.189|    0.180|    0.168|    0.178|    0.184|0.1827
|Ratio(Score / Mov)  |    0.106|    0.100|    0.100|    0.088|    0.094|    0.104|0.0987
|Ratio(Score / Fires)|    0.434|    0.402|    0.388|    0.355|    0.387|    0.415|0.3968
|Ratio(Score/F/M, %) |  0.00289|	0.00268|	0.00259|	0.00237|	0.00258|	0.00277|0.002647

After score training (5 test runs):

|                    |    1     |    2     |    3     |    4     |    5     |Average|
|--------------------|---------:|---------:|---------:|---------:|---------:|---:|
|Lives Consumed      | 1772.0000| 1743.0000| 1970.0000| 1800.0000| 1759.0000|1808.8
|Score               | 1415.0000| 1425.0000| 1565.0000| 1529.0000| 1471.0000|1481
|Fires               | 2780.0000| 3017.0000| 3281.0000| 3199.0000| 2909.0000|3037.2
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001
|Ratio(Lives / Mov)  |    0.1477|    0.1452|    0.1642|    0.1500|    0.1466|0.15074
|Ratio(Score / Mov)  |    0.1179|    0.1187|    0.1304|    0.1274|    0.1226|0.1234
|Ratio(Score / Fires)|    0.5090|    0.4723|    0.4770|    0.4780|    0.5057|0.4884
|Ratio(Score/F/M, %) |    0.0042|    0.0039|    0.0040|    0.0040|    0.0042|0.00406

**Observation:** On average the $R_{fm}$ increases 40%, $R_{f}$ increases 20%.

<p style="text-align: center;">**enemy frequency: 1 / 4**</p>

Before Score training (5 test runs):

|                    |    1     |    2     |    3    |    4    |    5     |Average|
|--------------------|---------:|---------:|--------:|--------:|---------:|---:|
|Lives Consumed      |  492.0000|  444.0000|  544.000|  557.000|  555.0000|518.4
|Score               |  291.0000|  289.0000|  319.000|  330.000|  294.0000|304.6
|Fires               | 3056.0000| 2980.0000| 3044.000| 3044.000| 3054.0000|3035.6
|Moves               |12001.0000|12001.0000|12001.000|12001.000|12001.0000|12001
|Ratio(Lives / Mov)  |    0.0410|    0.0370|    0.045|    0.046|    0.0462|0.04304
|Ratio(Score / Mov)  |    0.0242|    0.0241|    0.027|    0.027|    0.0245|0.02536
|Ratio(Score / Fires)|    0.0952|    0.0970|    0.105|    0.108|    0.0963|0.1003
|Ratio(Score/F/M, %) | 0.000793|	0.000808|	0.000875|	0.0009|	0.000802|0.0008356

After score training (5 test runs):

|                    |    1     |    2     |    3     |    4     |    5     |Average|
|--------------------|---------:|---------:|---------:|---------:|---------:|---:|
|Lives Consumed      |  469.0000|  429.0000|  373.0000|  400.0000|  425.0000|419.2
|Score               |  377.0000|  363.0000|  357.0000|  349.0000|  366.0000|362.4
|Fires               | 2041.0000| 1803.0000| 1549.0000| 1844.0000| 2331.0000|1913.6
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001
|Ratio(Lives / Mov)  |    0.0391|    0.0357|    0.0311|    0.0333|    0.0354|0.03492
|Ratio(Score / Mov)  |    0.0314|    0.0302|    0.0297|    0.0291|    0.0305|0.03018
|Ratio(Score / Fires)|    0.1847|    0.2013|    0.2305|    0.1893|    0.1570|0.19256
|Ratio(Score/F/M, %) |    0.0015|    0.0017|    0.0019|    0.0016|    0.0013|0.0016



**Observation:** On average the $R_{fm}$ increases 80%~100%, $R_{f}$ increases ~90%.

<p style="text-align: center;">**enemy frequency: 1 / 8**</p>

Before Score training (5 test runs):

|                    |    1     |    2     |    3     |    4     |    5     |Average|
|--------------------|---------:|---------:|---------:|---------:|---------:|---:|
|Lives Consumed      |  186.0000|  272.0000|  293.0000|  294.0000|  282.0000|265.4
|Score               |  124.0000|  188.0000|  146.0000|  172.0000|  167.0000|159.4
|Fires               | 3071.0000| 2874.0000| 2966.0000| 2936.0000| 2988.0000|2967
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001
|Ratio(Lives / Mov)  |    0.0155|    0.0227|    0.0244|    0.0245|    0.0235|0.02212
|Ratio(Score / Mov)  |    0.0103|    0.0157|    0.0122|    0.0143|    0.0139|0.01328
|Ratio(Score / Fires)|    0.0404|    0.0654|    0.0492|    0.0586|    0.0559|0.0539
|Ratio(Score/F/M, %) |0.000337|	0.000545|	0.000410|	0.000488|	0.000466|0.0004492

After score training (5 test runs):

|                    |    1     |    2     |    3     |    4     |    5     |Average|
|--------------------|---------:|---------:|---------:|---------:|---------:|---:|
|Lives Consumed      |  197.0000|  237.0000|  219.0000|  186.0000|  215.0000|210.8
|Score               |  171.0000|  180.0000|  198.0000|  176.0000|  182.0000|181.4
|Fires               | 1341.0000| 1585.0000| 2299.0000| 1314.0000| 1828.0000|1673.4
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001
|Ratio(Lives / Mov)  |    0.0164|    0.0197|    0.0182|    0.0155|    0.0179|0.01754
|Ratio(Score / Mov)  |    0.0142|    0.0150|    0.0165|    0.0147|    0.0152|0.01512
|Ratio(Score / Fires)|    0.1275|    0.1136|    0.0861|    0.1339|    0.0996|0.11214
|Ratio(Score/F/M, %) |    0.0011|    0.0009|    0.0007|    0.0011|    0.0008|0.00092


**Observation:** On average the $R_{fm}$ increases > 100%, $R_{f}$ increases > 100%.

### 4.2 Using survival training

<p style="text-align: center;">**enemy frequency: 1 / 1**</p>

Before Score training:

|                    |    1     |    2     |    3     |    4     |    5     |Average|
|--------------------|---------:|---------:|---------:|---------:|---------:|---:|
|Lives Consumed      | 1729.0000| 1675.0000| 1777.0000| 1802.0000| 1588.0000|1714.2
|Score               | 1306.0000| 1298.0000| 1325.0000| 1316.0000| 1307.0000|1310.4
|Fires               | 2920.0000| 2952.0000| 2936.0000| 3005.0000| 2983.0000|2959.2
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001
|Ratio(Lives / Mov)  |    0.1441|    0.1396|    0.1481|    0.1502|    0.1323|0.14286
|Ratio(Score / Mov)  |    0.1088|    0.1082|    0.1104|    0.1097|    0.1089|0.1092
|Ratio(Score / Fires)|    0.4473|    0.4397|    0.4513|    0.4379|    0.4381|0.44286
|Ratio(Score/F/M, %) |    0.0037|    0.0037|    0.0038|    0.0036|    0.0037|0.0037

After score training:

|                    |    1     |    2     |    3     |    4     |    5     | Average|
|--------------------|---------:|---------:|---------:|---------:|---------:|---:|
|Lives Consumed      | 1309.0000| 1261.0000| 1374.0000| 1375.0000| 1317.0000|1327.2|
|Score               | 1405.0000| 1372.0000| 1370.0000| 1433.0000| 1444.0000|1404.8|
|Fires               | 3300.0000| 3341.0000| 3262.0000| 3339.0000| 3342.0000|3316.8|
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001|
|Ratio(Lives / Mov)  |    0.1091|    0.1051|    0.1145|    0.1146|    0.1097|0.1106|
|Ratio(Score / Mov)  |    0.1171|    0.1143|    0.1142|    0.1194|    0.1203|0.11706|
|Ratio(Score / Fires)|    0.4258|    0.4107|    0.4200|    0.4292|    0.4321|0.42356|
|Ratio(Score/F/M, %) |    0.0035|    0.0034|    0.0035|    0.0036|    0.0036|0.00352|

**Observation:** The improvements during survival training at high enemey frequency is low, because there is too many enemies. The stragety is forced to shoot fires constantly. And even the new stragety actually has worse performance than randomly shooting fires. But lives consumed is still lower.

<p style="text-align: center;">**enemy frequency: 1 / 4**</p>

Before Score training:

|                    |    1     |    2     |    3     |    4     |    5     | average  |
|--------------------|---------:|---------:|---------:|---------:|---------:|---------:|
|Lives Consumed      |  261.0000|  288.0000|  233.0000|  260.0000|  266.0000|  261.6000|
|Score               |  342.0000|  353.0000|  323.0000|  340.0000|  336.0000|  338.8000|
|Fires               | 3036.0000| 3047.0000| 3054.0000| 2967.0000| 3056.0000| 3032.0000|
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|
|Ratio(Lives / Mov)  |    0.0217|    0.0240|    0.0194|    0.0217|    0.0222|    0.0218|
|Ratio(Score / Mov)  |    0.0285|    0.0294|    0.0269|    0.0283|    0.0280|    0.0282|
|Ratio(Score / Fires)|    0.1126|    0.1159|    0.1058|    0.1146|    0.1099|    0.1118|
|Ratio(Score/F/M, %) |    0.0009|    0.0010|    0.0009|    0.0010|    0.0009|    0.0009|

After score training:

|                    |    1     |    2     |    3     |    4     |    5     | Average|
|--------------------|---------:|---------:|---------:|---------:|---------:| -------:|
|Lives Consumed      |  213.0000|  255.0000|  211.0000|  227.0000|  195.0000|220.2|
|Score               |  347.0000|  395.0000|  358.0000|  389.0000|  365.0000|370.8|
|Fires               | 2742.0000| 2700.0000| 2762.0000| 2687.0000| 2735.0000|2725.2|
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001|
|Ratio(Lives / Mov)  |    0.0177|    0.0212|    0.0176|    0.0189|    0.0162|0.01832|
|Ratio(Score / Mov)  |    0.0289|    0.0329|    0.0298|    0.0324|    0.0304|0.03088|
|Ratio(Score / Fires)|    0.1265|    0.1463|    0.1296|    0.1448|    0.1335|0.13614|
|Ratio(Score/F/M, %) |    0.0011|    0.0012|    0.0011|    0.0012|    0.0011|0.00114|


**Observation:** On average the $R_{fm}$ increases only 10%, $R_{f}$ increases 30%. The resason is probably beause there arestill too many enemies. Further decreasing the enemy amounts might increase the performance.

<p style="text-align: center;">**enemy frequency: 1 / 8**</p>

Before Score training:

|                    |    1     |    2     |    3     |    4     |    5     | average  |
|--------------------|---------:|---------:|---------:|---------:|---------:|---------:|
|Lives Consumed      |  147.0000|  123.0000|  145.0000|  104.0000|  110.0000|  125.8000|
|Score               |  169.0000|  175.0000|  171.0000|  173.0000|  160.0000|  169.6000|
|Fires               | 2985.0000| 2993.0000| 2969.0000| 3075.0000| 3005.0000| 3005.4000|
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|
|Ratio(Lives / Mov)  |    0.0122|    0.0102|    0.0121|    0.0087|    0.0092|    0.0105|
|Ratio(Score / Mov)  |    0.0141|    0.0146|    0.0142|    0.0144|    0.0133|    0.0141|
|Ratio(Score / Fires)|    0.0566|    0.0585|    0.0576|    0.0563|    0.0532|    0.0564|
|Ratio(Score/F/M, %) |    0.0005|    0.0005|    0.0005|    0.0005|    0.0004|    0.0005|

After score training:

|                    |    1     |    2     |    3     |    4     |    5     |  average  |
|--------------------|---------:|---------:|---------:|---------:|---------:|----------:|
|Lives Consumed      |  104.0000|  116.0000|   95.0000|  107.0000|  105.0000|  105.40000|
|Score               |  173.0000|  188.0000|  199.0000|  188.0000|  178.0000|  185.20000|
|Fires               | 2413.0000| 2317.0000| 2452.0000| 2513.0000| 2564.0000| 2451.80000|
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|12001.00000|
|Ratio(Lives / Mov)  |    0.0087|    0.0097|    0.0079|    0.0089|    0.0087|    0.00878|
|Ratio(Score / Mov)  |    0.0144|    0.0157|    0.0166|    0.0157|    0.0148|    0.01543|
|Ratio(Score / Fires)|    0.0717|    0.0811|    0.0812|    0.0748|    0.0694|    0.07565|
|Ratio(Score/F/M, %) |    0.0006|    0.0007|    0.0007|    0.0006|    0.0006|    0.00063|


**Observation:** On average the $R_{fm}$ still increases only 20%, $R_{f}$ increases 40%. The combination of two strategies doesn't work too well.